In [83]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
## Fundamental Data Analysis
funda_data = pd.read_csv('Fundamental_data.csv')
funda_data.columns

In [ ]:
funda_data = funda_data.drop(columns=['public_date',"pcf",'staff_sale','PEG_trailing','gvkey','permno','adate'])
funda_data = funda_data.drop_duplicates()

In there, we delete the unuseful columns named pulic_date, pcf, staff_sale, peg_trailing, gvkey, permno and drop the duplicates for each rows. We also delete the adate data out of the purpose to delete the unuseful information here. Finally, we gonna use the qdate to be the corresponding time to mark the ticker.

### Daily Indicators

In [15]:
for name in pd.unique(funda_data.TICKER).tolist():
    funda_stock = funda_data.loc[funda_data['TICKER'] == name,:]
    

MSFT
nan
FB
AAPL
BRK
JNJ
AMZN
NVDA
GOOG
GOOGL
V
TSLA


In [16]:
funda_data.loc[funda_data['TICKER'] == 'TSLA',:]

,qdate,dpr,npm,gpm,roa,roe,capital_ratio,de_ratio,cash_ratio,curr_ratio,inv_turn,pay_turn,sale_nwc,rd_sale,accrual,TICKER
1051,06/30/2010,NaN,-0.855,0.248,-0.552,NaN,0.402,2.682,0.964,1.572,3.144,15.549,3.208,0.336,-0.060,TSLA
1054,09/30/2010,NaN,-1.284,0.312,-0.768,NaN,0.287,1.344,1.427,2.091,2.257,15.549,1.426,0.641,-0.049,TSLA
1057,12/31/2010,NaN,-1.322,0.354,-0.527,NaN,0.259,0.865,2.024,2.757,2.204,4.421,0.777,0.797,-0.103,TSLA
1060,03/31/2011,NaN,-1.199,0.393,-0.600,-0.484,0.315,1.157,1.631,2.359,2.134,3.362,1.268,0.834,-0.103,TSLA
1063,06/30/2011,NaN,-1.111,0.406,-0.558,-0.811,0.280,0.910,2.046,2.759,2.186,3.158,0.977,0.904,-0.117,TSLA
1066,09/30/2011,NaN,-1.115,0.398,-0.460,-0.951,0.345,1.103,1.996,2.630,2.427,2.761,1.004,0.922,-0.205,TSLA
1069,12/31/2011,NaN,-1.246,0.384,-0.427,-2.271,0.548,2.184,1.588,1.949,2.640,3.070,1.125,1.023,-0.255,TSLA
1072,03/31/2012,NaN,-1.593,0.385,-0.446,-1.142,0.489,1.764,1.737,2.169,2.182,2.052,0.881,1.274,-0.270,TSLA
1075,06/30/2012,NaN,-2.223,0.375,-0.458,-1.341,0.628,3.020,1.328,1.684,1.739,1.719,1.037,1.680,-0.253,TSLA
1078,09/30/2012,NaN,-2.651,0.238,-0.493,-2.112,0.777,6.424,0.844,1.213,1.346,2.451,2.501,1.821,-0.206,TSLA


### Weekly Indicators

##  MarcoEconomic Anslysis


In [84]:
macro = pd.read_csv("macro_data_1.csv")
macro = macro.loc[macro.datadate < '2020-02-01']

#macro.to_csv("macro_data.csv",index=0)




### 3. Macroeconomic Data
- REAL_GDP: annual data type 
- REAL_GDP_PER_CAPITA: quarterly data type 
- TREASURY_YIELD: short time(3-month: risk free rate)/ long time (30 years) 
- FEDERAL_FUNDS_RATE: Board of Governors of the Federal Reserve System (US), Federal Funds Effective Rate, retrieved from FRED, Federal Reserve Bank of St. Louis
- CPI: monthly data 
- INFLATION: annual data 
- UNEMPLOYMENT: The unemployment rate represents the number of unemployed as a percentage of the labor force.  
- NONFARM_PAYROLL: the monthly US All Employees, a measure of the number of U.S. workers in the economy that excludes proprietors, private household employees, unpaid volunteers, farm employees, and the unincorporated self-employed.

In [94]:
def moving_normal(series,win):
    new = (series - series.rolling(window=win).min())/(series.rolling(window=win).max() - series.rolling(window=win).min())
   
    return new

In [95]:
macro_columns = ['datadate','gdp','gdpr1','gdpr2','cpi','bond20yr','bond30yr','fedfunds','cpir','wpir','unemp','employ']
macro_use = macro[macro_columns]
stock_list = ['AAPL','MSFT','GOOG','AMZN','NVDA','BRK-B','TSLA','META','JNJ','V']
macro_use.iloc[:,2:3].apply(lambda x: moving_normal(x, win=4),axis=0).head(40)

,gdpr1
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


### Fill with 0


In [ ]:
AAPL = pd.read_csv("../stock_price/Daily/AAPL.csv")
AAPL = pd.merge(AAPL[['Date','adjusted_close']],macro_use, left_on='Date', right_on='datadate',how='outer')
AAPL.loc[AAPL.Date.isna(),'Date'] = AAPL.loc[AAPL.Date.isna(),'datadate']
AAPL = AAPL.sort_values(by='Date').drop('datadate',axis=1)
macro_list = ['gdp','gdpr1','gdpr2','cpi','bond20yr','bond30yr','fedfunds','cpir','wpir','unemp','employ']

AAPL.loc[:,macro_list] = AAPL.loc[:,macro_list].bfill()
AAPL.head(40)


In [50]:
import os
import sys

if not os.path.isdir('Macro'):
    os.mkdir('Macro')
if not os.path.isdir('Macro/Daily'):
    os.mkdir('Macro/Daily')
if not os.path.isdir('Macro/Weekly'):
    os.mkdir('Macro/Weekly')

if not os.path.isdir('Macro/Monthly'):
    os.mkdir('Macro/Monthly')


macro_bfill_daily = {}


for stock in stock_list:
    stock_df = pd.read_csv("../encode_price/Daily/"+stock+".csv")
    stock_df = pd.merge(stock_df[['Date','5. adjusted close']],macro_use, left_on='Date', right_on='datadate',how='outer')
    stock_df.loc[stock_df.Date.isna(),'Date'] = stock_df.loc[stock_df.Date.isna(),'datadate']
    stock_df = stock_df.sort_values(by='Date').drop('datadate',axis=1)
    stock_df.rename(columns={"5. adjusted close":"adjusted_close"},inplace=True)
    stock_df.loc[:,macro_list] = stock_df.loc[:,macro_list].bfill()
    macro_bfill_daily[stock] = stock_df
    stock_df.dropna(subset=['adjusted_close'],inplace=True)
    stock_df.to_csv("Macro/Daily/"+stock+".csv",index=0)




In [ ]:
macro_bfill_daily['AAPL'].head(40)

In [52]:
macro_bfill_weekly = {}

for stock in stock_list:
    stock_df = pd.read_csv("../stock_price/Weekly/"+stock+".csv")
    stock_df = pd.merge(stock_df[['Date','adjusted_close']],macro_use, left_on='Date', right_on='datadate',how='outer')
    stock_df.loc[stock_df.Date.isna(),'Date'] = stock_df.loc[stock_df.Date.isna(),'datadate']
    stock_df = stock_df.sort_values(by='Date').drop('datadate',axis=1)
    stock_df.rename(columns={"5. adjusted close":"adjusted_close"},inplace=True)
    stock_df.loc[:,macro_list] = stock_df.loc[:,macro_list].bfill()
    stock_df.dropna(subset=['adjusted_close'],inplace=True)
    macro_bfill_weekly[stock] = stock_df
    stock_df.to_csv("Macro/Weekly/"+stock+".csv",index=0)




In [57]:
macro_bfill_monthly = {}

for stock in stock_list:
    stock_df = pd.read_csv("../stock_price/Monthly/"+stock+".csv")
    stock_df = pd.merge(stock_df[['Date','adjusted_close']],macro_use, left_on='Date', right_on='datadate',how='outer')
    stock_df.loc[stock_df.Date.isna(),'Date'] = stock_df.loc[stock_df.Date.isna(),'datadate']
    stock_df = stock_df.sort_values(by='Date').drop('datadate',axis=1)
    stock_df.rename(columns={"5. adjusted close":"adjusted_close"},inplace=True)
    stock_df.loc[:,macro_list] = stock_df.loc[:,macro_list].bfill()
    macro_bfill_monthly[stock] = stock_df
    stock_df.dropna(subset=['adjusted_close'],inplace=True)
    stock_df.to_csv("Macro/Monthly/"+stock+".csv",index=0)



In [ ]:
macro_bfill_monthly['AAPL'].head(40)

### Technical Alpha Factor Generation
- SMA: simple moving average of closing price
- EMA: exponential moving average of closing price
- VWAP: the volume weighted average for intra
- MACD: the movign average convergence/divergence values
- STOCH: the stochastic oscillator values where here the close is in relation to the recent trading range
- RSI: the relative strength index (RSI) values, where calculates a ratio of the recent upward price movements to the absolute price movement
- MFI: the money flow index values - look for divergence with price to signal reversals. 
- SAR: returns the parabolic SAR values
- AD: The Accumulation/Distribution Line is similar to the On Balance Volume (OBV), which sums the volume times +1/-1 based on whether the close is higher than the previous close.The Accumulation/Distribution Line is interpreted by looking for a divergence in the direction of the indicator relative to price. If the Accumulation/Distribution Line is trending upward it indicates that the price may follow. Also, if the Accumulation/Distribution Line becomes flat while the price is still rising (or falling) then it signals an impending flattening of the price.

In [8]:
from tech_indicators import *
import pandas as pd
if not os.path.isdir('Tech'):
    os.mkdir('Tech')
if not os.path.isdir('Tech/Daily'):
    os.mkdir('Tech/Daily')
if not os.path.isdir('Tech/Weekly'):
    os.mkdir('Tech/Weekly')

if not os.path.isdir('Tech/Monthly'):
    os.mkdir('Tech/Monthly')

stock_list = ['AAPL','MSFT','GOOG','AMZN','NVDA','BRK-B','TSLA','META','JNJ','V']

In [ ]:
AAPL = pd.read_csv("../stock_price/Daily/AAPL.csv")
df = AAPL.copy()
sma = calculate_sma(df, window=14)
ema = calculate_ema(df, window=14)
k, d = calculate_stoch(df)
rsi = calculate_rsi(df)
mfi = calculate_mfi(df)
sar = calculate_sar(df)
ad = calc_AD(df)

tech_Daily = pd.DataFrame({"Date":df.loc[:,"Date"],
              "SMA":sma,
              "EMA":ema,
              "STOCH_k":k,
              "STOCK_d":d,
              "RSI":rsi,
              "MFI":mfi,
              "SAR":sar,
              "AD":ad
              })

In [18]:
for stock in stock_list:
    stock_df = pd.read_csv("../stock_price/Daily/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date")
    sma = calculate_sma(df, window=14)
    ema = calculate_ema(df, window=14)
    k, d = calculate_stoch(df)
    rsi = calculate_rsi(df)
    mfi = calculate_mfi(df)
    sar = calculate_sar(df)
    ad = calc_AD(df)

    tech_Daily = pd.DataFrame({"Date":df.loc[:,"Date"],
              "SMA":sma,
              "EMA":ema,
              "STOCH_k":k,
              "STOCK_d":d,
              "RSI":rsi,
              "MFI":mfi,
              "SAR":sar,
              "AD":ad
              })
    tech_Daily.to_csv("Tech/Daily/"+stock+".csv",index=0)

In [19]:
for stock in stock_list:
    stock_df = pd.read_csv("../stock_price/Weekly/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date")
    sma = calculate_sma(df, window=5)
    ema = calculate_ema(df, window=5)
    k, d = calculate_stoch(df,k_window=5)
    rsi = calculate_rsi(df,period=5)
    mfi = calculate_mfi(df,period=5)
    sar = calculate_sar(df,lookback=5)
    ad = calc_AD(df)

    tech_Weekly = pd.DataFrame({"Date":df.loc[:,"Date"],
              "SMA":sma,
              "EMA":ema,
              "STOCH_k":k,
              "STOCK_d":d,
              "RSI":rsi,
              "MFI":mfi,
              "SAR":sar,
              "AD":ad
              })
    
    tech_Weekly.to_csv("Tech/Weekly/"+stock+".csv",index=0)

In [21]:
for stock in stock_list:
    stock_df = pd.read_csv("../stock_price/Monthly/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date")
    sma = calculate_sma(df, window=5)
    ema = calculate_ema(df, window=5)
    k, d = calculate_stoch(df,k_window=5)
    rsi = calculate_rsi(df,period=5)
    mfi = calculate_mfi(df,period=5)
    sar = calculate_sar(df,lookback=5)
    ad = calc_AD(df)

    tech_Monthly = pd.DataFrame({"Date":df.loc[:,"Date"],
              "SMA":sma,
              "EMA":ema,
              "STOCH_k":k,
              "STOCK_d":d,
              "RSI":rsi,
              "MFI":mfi,
              "SAR":sar,
              "AD":ad
              })
    
    tech_Monthly.to_csv("Tech/Monthly/"+stock+".csv",index=0)

### Merge Indicators

- Higham

In [48]:
## higham
def P_s(a, wh):
    # helper function for higham, the second projection
 
    a = wh.dot(a).dot(wh)
    vals, vecs = np.linalg.eigh(a)
    vals[np.where(vals < 0)] = 0
    a = vecs.dot(np.diag(vals)).dot(vecs.T)

    wh_inv = np.diag(1 / np.diagonal(wh))

    a = wh_inv.dot(a).dot(wh_inv)
    return a


def P_u(a):
    # helper function for higham, the first projection
    np.fill_diagonal(a, 1)
    return a


def higham_psd(a, tol=None, max_iter=100, weights=None):
    # higham nearest psd
    if tol is None:
        tol = np.spacing(1) * len(a)
    if weights is None:
        weights = np.ones(len(a))
    w_h = np.diag(np.sqrt(weights))
    Y = np.copy(a)
    ds = np.zeros(np.shape(a))
    for i in range(0, max_iter):
        norm_Y_pre = fro_norm(Y)
        R = Y - ds
        X = P_s(R, w_h)

        ds = X - R
        Y = P_u(X)
        norm_Y = fro_norm(Y)

        if -1 * tol < norm_Y - norm_Y_pre < tol:
            break
    return pd.DataFrame(Y)

def fro_norm(a):
    return np.sum(np.square(a))


In [74]:
def moving_normal(series,win):
    new = (series - series.rolling(window=win).mean())/series.rolling(window=win).std()
    return new

In [80]:
AAPL = pd.read_csv("Merged/Daily/AAPL.csv")
AAPL = AAPL.loc[(AAPL.Date<'2019-10-01')&(AAPL.Date>"2010-01-21")]
#AAPL.iloc[:,1:] = AAPL.iloc[:,1:].apply(lambda x:moving_normal(series=x,win=10),axis=0)
corr = AAPL.drop("Date",axis=1).corr()
higham = higham_psd(AAPL.drop("Date",axis=1).corr())


In [77]:
print(fro_norm(higham.values - corr.values))

1.6888769411126296e-28
